In [9]:
import os
import pandas as pd
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [10]:
# Define the root directory of the project as credit-mlops
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(ROOT_DIR)

/workspaces/credit-mlops


In [11]:
# Read the processed data
df = pd.read_csv(os.path.join(ROOT_DIR, 'data', 'processed', 'processed_data.csv'))
df.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0,2,1000.0,201.802084,139.509787,0.000000,12
1,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4,0,7000.0,4103.032597,1072.340217,0.222222,12


## Normalizing data

In [12]:
from sklearn.preprocessing import StandardScaler

# Create the MinMaxScaler object
scaler = StandardScaler()

# Fit and transform the data
df = scaler.fit_transform(df)

## Autoencoder

### Implement autoencoder

In [13]:
# Define the number of features
input_dim = df.shape[1]
encoding_dim = 10

# Define the input layer
input_layer = Input(shape=(input_dim, ))

# Define the encoder layer
encoder_layer = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer)

# Define the decoder layer
decoder_layer = Dense(input_dim, activation="relu")(encoder_layer)

# Define the autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoder_layer)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=1, mode='min', restore_best_weights=True)

# Train the autoencoder model
autoencoder.fit(df, df, epochs=100, batch_size=32, shuffle=True, validation_split=0.2, verbose=1, callbacks=[early_stopping])

# Save the autoencoder model
autoencoder.save(os.path.join(ROOT_DIR, 'models', 'autoencoder.pkl'))

Epoch 1/100


184/184 [==============================] - 1s 2ms/step - loss: 0.8916 - val_loss: 0.5545
Epoch 2/100
184/184 [==============================] - 0s 1ms/step - loss: 0.6524 - val_loss: 0.4521
Epoch 3/100
184/184 [==============================] - 0s 2ms/step - loss: 0.5581 - val_loss: 0.4152
Epoch 4/100
184/184 [==============================] - 0s 1ms/step - loss: 0.5065 - val_loss: 0.3942
Epoch 5/100
184/184 [==============================] - 0s 1ms/step - loss: 0.4687 - val_loss: 0.3782
Epoch 6/100
184/184 [==============================] - 0s 1ms/step - loss: 0.4374 - val_loss: 0.3655
Epoch 7/100
184/184 [==============================] - 0s 1ms/step - loss: 0.4112 - val_loss: 0.3545
Epoch 8/100
184/184 [==============================] - 0s 1ms/step - loss: 0.3915 - val_loss: 0.3472
Epoch 9/100
184/184 [==============================] - 0s 2ms/step - loss: 0.3760 - val_loss: 0.3416
Epoch 10/100
184/184 [==============================] - 0s 1ms/step - loss: 0.3635 - val_loss: 0.3370
E

INFO:tensorflow:Assets written to: /workspaces/credit-mlops/models/autoencoder.pkl/assets


### Apply the encoder model to get the reduced dimension

In [14]:
# Get the encoder model
encoder_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=1).output)

# Get the reduced data using the encoder
reduced_data = encoder_model.predict(df)

# Convert the reduced data to a DataFrame
reduced_df = pd.DataFrame(reduced_data, columns=[f"latent_{i+1}" for i in range(encoding_dim)])

# Save the reduced dataset to a CSV file
reduced_df.to_csv(os.path.join(ROOT_DIR, 'data', 'processed', 'featurized_data.csv'), index=False)

230/230 [==============================] - 0s 766us/step


In [15]:
# Check the shape of the reduced data
print(reduced_df.shape)

(7346, 10)
